In [8]:
import json
import requests
from requests.exceptions import HTTPError, ConnectionError, Timeout, RequestException
#add Path
from pathlib import Path
from datetime import datetime
import time
import random
import logging
import os

https://cdn.mcdonalds.pl/uploads/20250910144011/352978-tabela-wo-8-11-2023-mop.pdf?openOutsideMcd=true

In [9]:
pdf_PATH = Path("./pdf")
pdf_PATH.mkdir(parents=True, exist_ok=True)
print(pdf_PATH.exists())

True


In [10]:
os.makedirs("logs", exist_ok=True)
LOGS_PATH = Path("logs")
LOGS_FILE = LOGS_PATH / 'logs.log'
print(LOGS_PATH.exists())

True


In [11]:
logging.basicConfig(
    filename=str(LOGS_FILE),  # will create the logs folder (if it doesn't exist) and the errors.log file | str ensures compatibility with older Python
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

logger = logging.getLogger(__name__)

In [ ]:
BASE_URL = 'https://cdn.mcdonalds.pl/uploads/20250910144011/352978-tabela-wo-8-11-2023-mop.pdf?openOutsideMcd=true'

def scrape_pdf(url):
    try:
        url = BASE_URL
        response = requests.get(
            url,
            # params=params,
            headers={"version":"2"},

            #timeout for 5 sek
            timeout=5
        )


        response.raise_for_status()

        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        pdf_name=f'mcd_nutritional_table-time-{timestamp}.pdf'

        with open(pdf_PATH/pdf_name, 'wb') as f:
            f.write(response.content)
        logger.info(f'💚 udało się pobrać plik pfd {pdf_name}')
    except HTTPError as e:
        if 400 <= response.status_code < 500:
            logger.error(f'❌Błąd klienta {response.status_code}:{response.reason} dla strony..')
            return
        elif 500 <= response.status_code < 600:
            logger.warning(f'⚠️Błąd serwera {response.status_code}:{response.reason}')
        else:
            reason = getattr(response, 'reason', "brak opisu")
            logger.error(f'❌ Inny błąt HTTP {response.status_code}:{reason}')

    except (ConnectionAbortedError, Timeout) as e:
        logger.warning(f'🌐 Problem z połączeniem/timeout na stronie: {e}')
    
    except OSError as e:
        logger.error(f'❓ Niespodziewany wyjątek przy stronie, {e.__class__.__name__}: {e}')

    else:
    
        return None

In [45]:
scrape_pdf(BASE_URL)